In [1]:
import ee
import geemap

In [1]:
Map = geemap.Map()
Map

NameError: name 'geemap' is not defined

In [57]:
site = ee.FeatureCollection('users/Introduction/study_area')
Map.addLayer(site, {}, "study_area")

In [62]:
# Assignment 1

# Export the Night Lights images for May,2015 and May,2020
# Filter the collection to the date range
# Extract the 'avg_rad' band which represents the nighttime lights
# Clip the image to the geometry of your city
# Export the resulting image as a GeoTIFF file.

# Hint1:

# There are 2 VIIRS Nighttime Day/Night collections
# Use the one that corrects for stray light

# Hint2:

# The collection contains 1 global image per month
# After filtering for the month, there will be only 1 image in the collection

# You can use the following technique to extract that image
# image = ee.Image(filtered.first())

#==================================================================
#                 BEGINNING OF ASSIGNMENT 1 - MUKESH RAY
#==================================================================

# Loading and Filtering the study area from GAUL dataset -- in this case Delhi, India
delhi = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2") \
              .filter(ee.Filter.eq("ADM1_NAME", "Delhi"))



Map.centerObject(site, 9)
Map.addLayer(site)

# Loading the VIIRS image collection
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG")

# Filtering one year of VIIRS dataset for 2015 and taking the first image
night2015 = viirs.filter(ee.Filter.date("2015-01-01", "2015-12-31")) \
                      .select('avg_rad') \
                      .first()

# Filtering last one month of VIIRS dataset for 2020 (December)
night2020 = viirs.filter(ee.Filter.date("2020-12-01", "2020-12-31")) \
                      .select('avg_rad') \
                      .first()

# Visualizing the the datasets for 2015 and 2020
Map.addLayer(night2015.clip(site), {'min':0, 'max':50, 'palette': ['0c0c0c', '071aff', 'ff0000', 'ffbd03', 'fbff05', 'fffdfd']}, 'Night2015')
Map.addLayer(night2020.clip(site), {'min':0, 'max':50, 'palette': ['0c0c0c', '071aff', 'ff0000', 'ffbd03', 'fbff05', 'fffdfd']}, 'Night2020')

# Find the mean radiance in the Delhi for 2015 and 2020
mean2015 = night2015.reduceRegion(
  reducer= ee.Reducer.mean(),
  geometry = site,
  scale = 500,
  maxPixels = 1e9
  )

mean2020 = night2020.reduceRegion(
  reducer= ee.Reducer.mean(),
  geometry = site,
  scale = 500,
  maxPixels = 1e9
  )

print("Mean Radiance in 2015: ", mean2015.get('avg_rad').getInfo())
print("Mean Radiance in 2020: ", mean2020.get('avg_rad').getInfo())


# By comparing the mean of night light in 2015 and 2020, it is observed that the night light has decreased in 2020
# compared to 2015.

# # Export both the images
# Export.image.toDrive({
#   image: night2015,
#   description: "night2015",
#   folder: "GEE_Workshop",
#   region: delhi,
#   scale: 500,
#   maxPixels: 1e10,
#   fileFormat: 'GeoTIFF'
#   })

# Export.image.toDrive({
#   image: night2020,
#   description: "night2020",
#   folder: "GEE_Workshop",
#   region: delhi,
#   scale: 500,
#   maxPixels: 1e10,
#   fileFormat: 'GeoTIFF'
#   })

# END OF ASSIGNMENT 1


Mean Radiance in 2015:  5.32589382353451
Mean Radiance in 2020:  6.551199360512761


In [34]:
mu = ntl_tls.reduceRegion(reducer=ee.Reducer.mean())
std = ntl_tls.reduceRegion(reducer=ee.Reducer.stdDev())

# we'll cast these to native ee Numbers using the ee.Number constructor
mu = ee.Number(mu.get('avg_rad'))
std = ee.Number(std.get('avg_rad'))

# in order to confirm, we have to evaluate this to print client size (i.e. our computer)
print(f"the mean avg radiance (per month) for the pixels in our Timor-Leste region is: {mu.getInfo():.4f}")
print(f"the std dev is: {std.getInfo():.4f}")

the mean avg radiance (per month) for the pixels in our Timor-Leste region is: 0.2980
the std dev is: 0.6565
